In [8]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from pprint import pprint
from collections import Counter
import numpy as np
import pandas as pd
import pickle
import operator
import argparse
from os import listdir
from os.path import isfile, join
import talos
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
DATA_PATH = '../dataset/'

In [2]:
data = pd.read_csv("../../data/sample_data_intw.csv")

In [10]:
data.head(5)

,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,...,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,0,21408I70789,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,1539,...,6.0,0.0,2.0,12,6,0.0,29.000000,29.000000,UPW,20-07-2016
1,1,76462I70374,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,5787,...,12.0,0.0,1.0,12,12,0.0,0.000000,0.000000,UPW,10-08-2016
2,1,17943I70372,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,1539,...,6.0,0.0,1.0,6,6,0.0,0.000000,0.000000,UPW,19-08-2016
3,1,55773I70781,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,947,...,6.0,0.0,2.0,12,6,0.0,0.000000,0.000000,UPW,06-06-2016
4,1,03813I82730,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,2309,...,6.0,0.0,7.0,42,6,0.0,2.333333,2.333333,UPW,22-06-2016


In [11]:
#seperate categorical and numerical columns
non_num=[]
for i in data.columns:
    if data[i].dtype=="object":
        non_num.append(i)

print(non_num)

['msisdn', 'pcircle', 'pdate']


In [12]:
#drop these columns
data.drop(labels=non_num,axis=1,inplace=True)
#Prepare feature and target sets
x = data.drop(labels=["label"],axis=1)
y = data["label"]
x = talos.utils.rescale_meanzero(x)

#since our data is completely numeric, we can proceed for upsampling
from imblearn.over_sampling import SMOTE
#Initialise smote
smote = SMOTE()
# fit predictor and target variable on smote
x_smote, y_smote = smote.fit_resample(x,y)

print('old shape', x.shape)
print('new shape', x_smote.shape)

old shape (209593, 32)
new shape (366862, 32)


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_smote,y_smote,test_size=0.25,random_state=123)
print("length of x_train {} and y_train {}".format(len(x_train),len(y_train)))
print("length of x_test {} and y_test {}".format(len(x_test),len(y_test)))
# split for validation set
x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.20,random_state=123)
print("length of x_train {} and y_train {}".format(len(x_train),len(y_train)))
print("length of x_valid {} and y_valid {}".format(len(x_valid),len(y_valid)))


length of x_train 275146 and y_train 275146
length of x_test 91716 and y_test 91716
length of x_train 220116 and y_train 220116
length of x_valid 55030 and y_valid 55030


In [24]:
dataset_name = "tele"
pickle.dump(x_smote, open(DATA_PATH+dataset_name+'_features.p', 'wb'))
pickle.dump(y_smote, open(DATA_PATH+dataset_name+'_labels.p', 'wb'))